In [9]:
import pandas as pd
from fuzzywuzzy import fuzz
import swifter
from keras.models import Sequential
from keras.layers import Dense

In [10]:
train_df = pd.read_csv('../output/features.csv')
train_df.head()

,simple_ratio,partial_ratio,token_sort_ratio,token_set_ratio,is_duplicate
0,93,98,93,100,0
1,65,73,63,86,0
2,55,53,66,66,0
3,28,30,36,36,0
4,45,54,47,67,0


In [11]:
is_duplicate_df = train_df[['is_duplicate']].copy()
features_df = train_df.drop(['is_duplicate'], axis=1)

In [14]:
input_dim = len(features_df.columns)

model = Sequential()

model.add(Dense(12, input_dim=input_dim, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
model.fit(features_df, is_duplicate_df, epochs=5, batch_size=10)

Epoch 1/5
40429/40429 [==============================] - 21s 503us/step - loss: 0.5868 - accuracy: 0.6577
Epoch 2/5
40429/40429 [==============================] - 20s 492us/step - loss: 0.5472 - accuracy: 0.6646
Epoch 3/5
40429/40429 [==============================] - 20s 502us/step - loss: 0.5380 - accuracy: 0.6695
Epoch 4/5
40429/40429 [==============================] - 20s 502us/step - loss: 0.5367 - accuracy: 0.6713
Epoch 5/5
40429/40429 [==============================] - 20s 496us/step - loss: 0.5365 - accuracy: 0.6737
